# Clean


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
import os

from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))


c:\users\boog\dev\cfehome\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.4.0


In [2]:
def clean_text(text, remove_stopwords = True):
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text,  
                  flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    return(text)


In [3]:
os.getcwd()

'D:\\datascience\\capstone_nlp_bible\\seq2seq'

In [4]:
reviews = pd.read_csv("D:/datascience/capstone_nlp_bible/seq2seq/amazon-fine-food-reviews/Reviews.csv")

In [5]:
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
reviews.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [7]:
# Remove null values and unneeded features
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator',
                        'Score','Time'], 1)
reviews = reviews.reset_index(drop=True)


In [8]:
reviews.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


# Data Prep


In [9]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}


In [10]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return(text)


In [11]:
# remove stopwords, keepo for summary
# Clean the summaries and texts
clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")


Summaries are complete.
Texts are complete.


In [12]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1


In [13]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 132884


In [14]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('D:/datascience/capstone_nlp_bible/seq2seq/amazon-fine-food-reviews/numberbatch-en.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))


Word embeddings: 417195


In [15]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))


Number of words missing from CN: 3870
Percent of words that are missing from vocabulary: 2.91%


In [16]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))


Total number of unique words: 132884
Number of words we will use: 59595
Percent of words we will use: 44.85%


In [17]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))


59595


In [18]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count


In [19]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))


Total number of words in headlines: 25679933
Total number of UNKs in headlines: 192245
Percent of words that are UNK: 0.75%


In [20]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])


In [21]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())


Summaries:
              counts
count  568411.000000
mean        4.181624
std         2.657872
min         0.000000
25%         2.000000
50%         4.000000
75%         5.000000
max        48.000000

Texts:
              counts
count  568411.000000
mean       41.996835
std        42.520873
min         1.000000
25%        18.000000
50%        29.000000
75%        50.000000
max      2085.000000


In [22]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count


In [23]:
# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))


425615
425615


# Model building

In [58]:
def model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length


In [60]:
def process_encoding_input(target_data, vocab_to_int, batch_size):  
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1]) # slice it to target_data[0:batch_size, 0: -1]
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

# Create the encoding layers
bidirectional_dynamic_rnn use tf.variable_scope so that variables are reused with each layer

parameters

    -rnn_size: The number of units in the LSTM cell
    -sequence_length: size [batch_size], containing the actual lengths for each of the sequences in the batch
    -num_layers: number of bidirectional RNN layer
    -rnn_inputs: number of bidirectional RNN layer
    -keep_prob: RNN dropout input keep probability


In [61]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
            enc_output = tf.concat(enc_output,2)
            # original code is missing this line below, that is how we connect layers 
            # by feeding the current layer's output to next layer's input
            rnn_inputs = enc_output
    return enc_output, enc_state

# Create the training decoding layer
parameters

    -dec_embed_input: output of embedding_lookup for a batch of inputs
    -summary_length: length of each padded summary sequences in batch, since padded, all lengths should be same number
    -dec_cell: the decoder RNN cells' output with attention wapper
    -output_layer: fully connected layer to apply to the RNN output
    -vocab_size: vocabulary size i.e. len(vocab_to_int)+1
    -max_summary_length: the maximum length of a summary in a batch
    -batch_size: number of input sequences in a batch

Three components

    -TraingHelper reads a sequence of integers from the encoding layer.
    -BasicDecoder processes the sequence with the decoding cell, and an output layer, which is a fully connected layer. initial_state set to zero state.
    -dynamic_decode creates our outputs that will be used for training.


In [62]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, output_layer,
                            vocab_size, max_summary_length,batch_size):
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,
                                                       helper=training_helper,
                                                       initial_state=dec_cell.zero_state(dtype=tf.float32, batch_size=batch_size),
                                                       output_layer = output_layer)

    training_logits = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits


# Create infer decoding layer
parameters

    -embeddings: the CN's word_embedding_matrix
    -start_token: the id of <GO>
    -end_token: the id of <EOS>
    -dec_cell: the decoder RNN cells' output with attention wapper
    -output_layer: fully connected layer to apply to the RNN output
    -max_summary_length: the maximum length of a summary in a batch
    -batch_size: number of input sequences in a batch

GreedyEmbeddingHelper argument start_tokens: int32 vector shaped [batch_size], the start tokens.



In [63]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        dec_cell.zero_state(dtype=tf.float32, batch_size=batch_size),
                                                        output_layer)
                
    inference_logits = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits


# Create Decoding layer
3 parts: decoding cell, attention, and getting our logits.

## Decoding Cell:
Just a two layer LSTM with dropout.

### Attention:
Using Bhadanau, since trains faster than Luong.

### AttentionWrapper applies the attention mechanism to our decoding cell.

parameters

    -dec_embed_input: output of embedding_lookup for a batch of inputs
    -embeddings: the CN's word_embedding_matrix
    -enc_output: encoder layer output, containing the forward and the backward rnn output
    -enc_state: encoder layer state, a tuple containing the forward and the backward final states of bidirectional rnn.
    -vocab_size: vocabulary size i.e. len(vocab_to_int)+1
    -text_length: the actual lengths for each of the input text sequences in the batch
    -summary_length: the actual lengths for each of the input summary sequences in the batch
    -max_summary_length: the maximum length of a summary in a batch
    -rnn_size: The number of units in the LSTM cell
    -vocab_to_int: vocab_to_int the dictionary
    -keep_prob: RNN dropout input keep probability
    -batch_size: number of input sequences in a batch
    -num_layers: number of decoder RNN layer


In [64]:
def lstm_cell(lstm_size, keep_prob):
    cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    return tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob = keep_prob)

def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length,
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    dec_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    output_layer = Dense(vocab_size,kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                     enc_output,
                                                     text_length,
                                                     normalize=False,
                                                     name='BahdanauAttention')
    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,attn_mech,rnn_size)
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input,summary_length,dec_cell,
                                                  output_layer,
                                                  vocab_size,
                                                  max_summary_length,
                                                  batch_size)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,
                                                    vocab_to_int['<GO>'],
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell,
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
    return training_logits, inference_logits


In [65]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size) #shape=(batch_size, senquence length) each seq start with index of<GO>
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    return training_logits, inference_logits


In [66]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]


In [67]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths


In [68]:
print("'<PAD>' has id: {}".format(vocab_to_int['<PAD>']))
sorted_summaries_samples = sorted_summaries[7:50]
sorted_texts_samples = sorted_texts[7:50]
pad_summaries_batch_samples, pad_texts_batch_samples, pad_summaries_lengths_samples, pad_texts_lengths_samples = next(get_batches(
    sorted_summaries_samples, sorted_texts_samples, 5))
print("pad summaries batch samples:\n\r {}".format(pad_summaries_batch_samples))


'<PAD>' has id: 59592
pad summaries batch samples:
 [[   13    28  1068  2803 59592 59592 59592 59592 59592 59592]
 [  359    79   380  2215    91  1149   102     0    70   188]
 [    1    61   102     4    50    30   233    93   690 59592]
 [    0   108 59592 59592 59592 59592 59592 59592 59592 59592]
 [    0   102   638 59592 59592 59592 59592 59592 59592 59592]]


# Training


In [69]:
# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.95


In [70]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits[0].rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits[0].sample_id, name='predictions')
    
    # Create the weights for sequence_loss, the sould be all True across since each batch is padded
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")
graph_location = "./graph"
print(graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(train_graph)


Graph is built.
./graph


# Training


In [71]:
# Subset the data for training
start = 200000
end = start + 50000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))


The shortest text length: 25
The longest text length: 31


In [72]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "./best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break


Epoch   1/100 Batch   20/781 - Loss:  5.147, Seconds: 4.18
Epoch   1/100 Batch   40/781 - Loss:  2.903, Seconds: 4.32
Epoch   1/100 Batch   60/781 - Loss:  2.863, Seconds: 3.90
Epoch   1/100 Batch   80/781 - Loss:  2.770, Seconds: 4.50
Epoch   1/100 Batch  100/781 - Loss:  2.831, Seconds: 4.38
Epoch   1/100 Batch  120/781 - Loss:  2.753, Seconds: 4.12
Epoch   1/100 Batch  140/781 - Loss:  2.664, Seconds: 4.32
Epoch   1/100 Batch  160/781 - Loss:  2.684, Seconds: 4.82
Epoch   1/100 Batch  180/781 - Loss:  2.612, Seconds: 4.72
Epoch   1/100 Batch  200/781 - Loss:  2.693, Seconds: 4.06
Epoch   1/100 Batch  220/781 - Loss:  2.707, Seconds: 4.36
Epoch   1/100 Batch  240/781 - Loss:  2.681, Seconds: 3.58
Average loss for this update: 2.913
New Record!
Epoch   1/100 Batch  260/781 - Loss:  2.542, Seconds: 4.36
Epoch   1/100 Batch  280/781 - Loss:  2.441, Seconds: 4.58
Epoch   1/100 Batch  300/781 - Loss:  2.546, Seconds: 4.62
Epoch   1/100 Batch  320/781 - Loss:  2.636, Seconds: 4.12
Epoch   

Epoch   4/100 Batch  300/781 - Loss:  1.680, Seconds: 4.10
Epoch   4/100 Batch  320/781 - Loss:  1.741, Seconds: 4.18
Epoch   4/100 Batch  340/781 - Loss:  1.986, Seconds: 3.96
Epoch   4/100 Batch  360/781 - Loss:  1.787, Seconds: 4.68
Epoch   4/100 Batch  380/781 - Loss:  1.778, Seconds: 4.66
Epoch   4/100 Batch  400/781 - Loss:  1.656, Seconds: 4.76
Epoch   4/100 Batch  420/781 - Loss:  1.622, Seconds: 4.54
Epoch   4/100 Batch  440/781 - Loss:  1.658, Seconds: 4.22
Epoch   4/100 Batch  460/781 - Loss:  1.824, Seconds: 4.68
Epoch   4/100 Batch  480/781 - Loss:  1.850, Seconds: 4.26
Epoch   4/100 Batch  500/781 - Loss:  1.735, Seconds: 4.80
Average loss for this update: 1.74
New Record!
Epoch   4/100 Batch  520/781 - Loss:  1.736, Seconds: 4.98
Epoch   4/100 Batch  540/781 - Loss:  1.616, Seconds: 3.92
Epoch   4/100 Batch  560/781 - Loss:  1.647, Seconds: 4.18
Epoch   4/100 Batch  580/781 - Loss:  1.751, Seconds: 4.18
Epoch   4/100 Batch  600/781 - Loss:  1.770, Seconds: 4.02
Epoch   4

Epoch   7/100 Batch  580/781 - Loss:  1.442, Seconds: 4.12
Epoch   7/100 Batch  600/781 - Loss:  1.471, Seconds: 4.26
Epoch   7/100 Batch  620/781 - Loss:  1.653, Seconds: 4.54
Epoch   7/100 Batch  640/781 - Loss:  1.471, Seconds: 4.78
Epoch   7/100 Batch  660/781 - Loss:  1.485, Seconds: 4.48
Epoch   7/100 Batch  680/781 - Loss:  1.332, Seconds: 4.28
Epoch   7/100 Batch  700/781 - Loss:  1.335, Seconds: 4.90
Epoch   7/100 Batch  720/781 - Loss:  1.389, Seconds: 4.56
Epoch   7/100 Batch  740/781 - Loss:  1.670, Seconds: 3.96
Epoch   7/100 Batch  760/781 - Loss:  1.632, Seconds: 4.21
Average loss for this update: 1.461
No Improvement.
Epoch   7/100 Batch  780/781 - Loss:  1.451, Seconds: 4.38
Epoch   8/100 Batch   20/781 - Loss:  1.661, Seconds: 4.24
Epoch   8/100 Batch   40/781 - Loss:  1.520, Seconds: 4.42
Epoch   8/100 Batch   60/781 - Loss:  1.498, Seconds: 3.86
Epoch   8/100 Batch   80/781 - Loss:  1.443, Seconds: 4.76
Epoch   8/100 Batch  100/781 - Loss:  1.489, Seconds: 4.12
Epoc

Epoch  11/100 Batch   80/781 - Loss:  1.233, Seconds: 4.60
Epoch  11/100 Batch  100/781 - Loss:  1.274, Seconds: 4.00
Epoch  11/100 Batch  120/781 - Loss:  1.171, Seconds: 4.02
Epoch  11/100 Batch  140/781 - Loss:  1.125, Seconds: 4.12
Epoch  11/100 Batch  160/781 - Loss:  1.173, Seconds: 4.40
Epoch  11/100 Batch  180/781 - Loss:  1.204, Seconds: 4.32
Epoch  11/100 Batch  200/781 - Loss:  1.263, Seconds: 3.88
Epoch  11/100 Batch  220/781 - Loss:  1.231, Seconds: 4.16
Epoch  11/100 Batch  240/781 - Loss:  1.166, Seconds: 3.54
Average loss for this update: 1.234
No Improvement.
Epoch  11/100 Batch  260/781 - Loss:  1.117, Seconds: 4.30
Epoch  11/100 Batch  280/781 - Loss:  1.009, Seconds: 4.66
Epoch  11/100 Batch  300/781 - Loss:  1.132, Seconds: 4.08
Epoch  11/100 Batch  320/781 - Loss:  1.165, Seconds: 4.22
Epoch  11/100 Batch  340/781 - Loss:  1.359, Seconds: 3.76
Epoch  11/100 Batch  360/781 - Loss:  1.237, Seconds: 4.58
Epoch  11/100 Batch  380/781 - Loss:  1.186, Seconds: 4.58
Epoc

Epoch  14/100 Batch  360/781 - Loss:  1.058, Seconds: 4.94
Epoch  14/100 Batch  380/781 - Loss:  1.012, Seconds: 4.78
Epoch  14/100 Batch  400/781 - Loss:  0.969, Seconds: 4.74
Epoch  14/100 Batch  420/781 - Loss:  0.925, Seconds: 3.96
Epoch  14/100 Batch  440/781 - Loss:  0.945, Seconds: 4.24
Epoch  14/100 Batch  460/781 - Loss:  1.020, Seconds: 4.26
Epoch  14/100 Batch  480/781 - Loss:  1.051, Seconds: 4.26
Epoch  14/100 Batch  500/781 - Loss:  1.005, Seconds: 4.68
Average loss for this update: 1.006
New Record!
Epoch  14/100 Batch  520/781 - Loss:  1.018, Seconds: 4.78
Epoch  14/100 Batch  540/781 - Loss:  0.899, Seconds: 3.80
Epoch  14/100 Batch  560/781 - Loss:  0.909, Seconds: 4.54
Epoch  14/100 Batch  580/781 - Loss:  0.964, Seconds: 4.32
Epoch  14/100 Batch  600/781 - Loss:  1.010, Seconds: 4.16
Epoch  14/100 Batch  620/781 - Loss:  1.142, Seconds: 4.84
Epoch  14/100 Batch  640/781 - Loss:  0.998, Seconds: 5.25
Epoch  14/100 Batch  660/781 - Loss:  1.031, Seconds: 4.86
Epoch  1

...

Epoch  21/100 Batch  140/781 - Loss:  0.694, Seconds: 4.12
Epoch  21/100 Batch  160/781 - Loss:  0.712, Seconds: 4.42
Epoch  21/100 Batch  180/781 - Loss:  0.746, Seconds: 4.32
Epoch  21/100 Batch  200/781 - Loss:  0.768, Seconds: 3.98
Epoch  21/100 Batch  220/781 - Loss:  0.736, Seconds: 4.54
Epoch  21/100 Batch  240/781 - Loss:  0.711, Seconds: 3.80
Average loss for this update: 0.758
No Improvement.
Epoch  21/100 Batch  260/781 - Loss:  0.670, Seconds: 4.14
Epoch  21/100 Batch  280/781 - Loss:  0.607, Seconds: 4.52
Epoch  21/100 Batch  300/781 - Loss:  0.718, Seconds: 4.10
Epoch  21/100 Batch  320/781 - Loss:  0.720, Seconds: 4.18
Epoch  21/100 Batch  340/781 - Loss:  0.801, Seconds: 3.80
Epoch  21/100 Batch  360/781 - Loss:  0.747, Seconds: 4.60
Epoch  21/100 Batch  380/781 - Loss:  0.705, Seconds: 4.68
Epoch  21/100 Batch  400/781 - Loss:  0.690, Seconds: 4.58
Epoch  21/100 Batch  420/781 - Loss:  0.642, Seconds: 4.06
Epoch  21/100 Batch  440/781 - Loss:  0.666, Seconds: 4.26
Epoc

Epoch  24/100 Batch  420/781 - Loss:  0.551, Seconds: 4.00
Epoch  24/100 Batch  440/781 - Loss:  0.569, Seconds: 4.38
Epoch  24/100 Batch  460/781 - Loss:  0.570, Seconds: 4.38
Epoch  24/100 Batch  480/781 - Loss:  0.613, Seconds: 4.20
Epoch  24/100 Batch  500/781 - Loss:  0.589, Seconds: 4.70
Average loss for this update: 0.6
New Record!
Epoch  24/100 Batch  520/781 - Loss:  0.641, Seconds: 4.52
Epoch  24/100 Batch  540/781 - Loss:  0.565, Seconds: 3.92
Epoch  24/100 Batch  560/781 - Loss:  0.555, Seconds: 4.14
Epoch  24/100 Batch  580/781 - Loss:  0.573, Seconds: 4.18
Epoch  24/100 Batch  600/781 - Loss:  0.586, Seconds: 3.96
Epoch  24/100 Batch  620/781 - Loss:  0.641, Seconds: 4.60
Epoch  24/100 Batch  640/781 - Loss:  0.573, Seconds: 4.78
Epoch  24/100 Batch  660/781 - Loss:  0.600, Seconds: 4.34
Epoch  24/100 Batch  680/781 - Loss:  0.533, Seconds: 4.12
Epoch  24/100 Batch  700/781 - Loss:  0.568, Seconds: 4.80
Epoch  24/100 Batch  720/781 - Loss:  0.603, Seconds: 4.42
Epoch  24/

Epoch  27/100 Batch  700/781 - Loss:  0.474, Seconds: 4.94
Epoch  27/100 Batch  720/781 - Loss:  0.519, Seconds: 4.56
Epoch  27/100 Batch  740/781 - Loss:  0.631, Seconds: 3.92
Epoch  27/100 Batch  760/781 - Loss:  0.608, Seconds: 3.98
Average loss for this update: 0.517
No Improvement.
Epoch  27/100 Batch  780/781 - Loss:  0.543, Seconds: 4.62
Epoch  28/100 Batch   20/781 - Loss:  0.643, Seconds: 4.26
Epoch  28/100 Batch   40/781 - Loss:  0.578, Seconds: 4.30
Epoch  28/100 Batch   60/781 - Loss:  0.546, Seconds: 3.84
Epoch  28/100 Batch   80/781 - Loss:  0.516, Seconds: 4.40
Epoch  28/100 Batch  100/781 - Loss:  0.507, Seconds: 4.00
Epoch  28/100 Batch  120/781 - Loss:  0.463, Seconds: 4.04
Epoch  28/100 Batch  140/781 - Loss:  0.455, Seconds: 4.04
Epoch  28/100 Batch  160/781 - Loss:  0.490, Seconds: 4.52
Epoch  28/100 Batch  180/781 - Loss:  0.531, Seconds: 4.34
Epoch  28/100 Batch  200/781 - Loss:  0.533, Seconds: 4.04
Epoch  28/100 Batch  220/781 - Loss:  0.507, Seconds: 4.00
Epoc

Epoch  31/100 Batch  200/781 - Loss:  0.461, Seconds: 3.90
Epoch  31/100 Batch  220/781 - Loss:  0.447, Seconds: 4.08
Epoch  31/100 Batch  240/781 - Loss:  0.426, Seconds: 3.52
Average loss for this update: 0.444
No Improvement.
Epoch  31/100 Batch  260/781 - Loss:  0.430, Seconds: 4.12
Epoch  31/100 Batch  280/781 - Loss:  0.380, Seconds: 4.56
Epoch  31/100 Batch  300/781 - Loss:  0.451, Seconds: 4.10
Epoch  31/100 Batch  320/781 - Loss:  0.424, Seconds: 4.20
Epoch  31/100 Batch  340/781 - Loss:  0.461, Seconds: 3.92
Epoch  31/100 Batch  360/781 - Loss:  0.437, Seconds: 4.76
Epoch  31/100 Batch  380/781 - Loss:  0.421, Seconds: 4.62
Epoch  31/100 Batch  400/781 - Loss:  0.421, Seconds: 4.56
Epoch  31/100 Batch  420/781 - Loss:  0.396, Seconds: 4.02
Epoch  31/100 Batch  440/781 - Loss:  0.408, Seconds: 4.20
Epoch  31/100 Batch  460/781 - Loss:  0.394, Seconds: 4.30
Epoch  31/100 Batch  480/781 - Loss:  0.425, Seconds: 4.16
Epoch  31/100 Batch  500/781 - Loss:  0.424, Seconds: 4.68
Aver

Epoch  34/100 Batch  480/781 - Loss:  0.374, Seconds: 4.26
Epoch  34/100 Batch  500/781 - Loss:  0.360, Seconds: 4.64
Average loss for this update: 0.364
New Record!
Epoch  34/100 Batch  520/781 - Loss:  0.395, Seconds: 4.46
Epoch  34/100 Batch  540/781 - Loss:  0.339, Seconds: 3.94
Epoch  34/100 Batch  560/781 - Loss:  0.341, Seconds: 4.06
Epoch  34/100 Batch  580/781 - Loss:  0.342, Seconds: 4.14
Epoch  34/100 Batch  600/781 - Loss:  0.358, Seconds: 4.00
Epoch  34/100 Batch  620/781 - Loss:  0.365, Seconds: 4.52
Epoch  34/100 Batch  640/781 - Loss:  0.334, Seconds: 4.74
Epoch  34/100 Batch  660/781 - Loss:  0.363, Seconds: 4.42
Epoch  34/100 Batch  680/781 - Loss:  0.310, Seconds: 4.60
Epoch  34/100 Batch  700/781 - Loss:  0.329, Seconds: 4.72
Epoch  34/100 Batch  720/781 - Loss:  0.358, Seconds: 4.40
Epoch  34/100 Batch  740/781 - Loss:  0.429, Seconds: 4.04
Epoch  34/100 Batch  760/781 - Loss:  0.410, Seconds: 4.16
Average loss for this update: 0.359
New Record!
Epoch  34/100 Batch

# Summaries of our own

In [75]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]


In [76]:
input_sentences=["The coffee tasted great and was at such a good price! I highly recommend this to everyone!",
               "love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets"]
generagte_summary_length =  [3,2]

texts = [text_to_seq(input_sentence) for input_sentence in input_sentences]
checkpoint = "./best_model.ckpt"
if type(generagte_summary_length) is list:
    if len(input_sentences)!=len(generagte_summary_length):
        raise Exception("[Error] makeSummaries parameter generagte_summary_length must be same length as input_sentences or an integer")
    generagte_summary_length_list = generagte_summary_length
else:
    generagte_summary_length_list = [generagte_summary_length] * len(texts)
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    #Multiply by batch_size to match the model's input parameters
    for i, text in enumerate(texts):
        generagte_summary_length = generagte_summary_length_list[i]
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                          summary_length: [generagte_summary_length], #summary_length: [np.random.randint(5,8)], 
                                          text_length: [len(text)]*batch_size,
                                          keep_prob: 1.0})[0] 
        # Remove the padding from the summaries
        pad = vocab_to_int["<PAD>"] 
        print('- Review:\n\r {}'.format(input_sentences[i]))
        print('- Summary:\n\r {}\n\r\n\r'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))


INFO:tensorflow:Restoring parameters from ./best_model.ckpt
- Review:
 The coffee tasted great and was at such a good price! I highly recommend this to everyone!
- Summary:
 good coffee good


- Review:
 love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets
- Summary:
 best coffee


